In [ ]:
import numpy as np

In [ ]:
def fit(X_train,y_train):
    result = {}
    class_values = set(y_train)       #Gives distinct Class Values
    
    for current_class in class_values:
        result[current_class] = {}
        result['total_data'] = len(y_train)
        
        #Getting Rows of a particular Class
        current_class_rows = (y_train == current_class)                    #Boolean Array
        X_train_current = X_train[current_class_rows]
        y_train_current = y_train[current_class_rows]

        result[current_class]["total_count"] = len(y_train_current)
        
        num_features = X_train.shape[1]        
        for j in range(1,num_features+1):              #instead of 0,1,2,3,4 we want 1,2,3,4,5   
            result[current_class][j] = {}                                                   
                                                                                            
            all_possible_values = set(X_train[:,j-1])                  #x^j           #j-1 because starting from 1
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train[:,j-1] == current_value).sum()           #Boolean,sum=count of current value
                
                
                
        return result 
             

In [ ]:
def probability(dictionary,x,current_class):
    #output = 1
    #class_prob = dictionary[current_class][total_count] / dictionary['total_data']
    
    #output = dictionary[current_class][total_count] / dictionary['total_data']
    output = np.log(dictionary[current_class]['total_count']) - np.log(dictionary['total_data'])
    
    
    num_features = len(dictionary[current_class].keys()) - 1                  #One is total_count
    for j in range(1,num_features+1):
        xj = x[j-1]
        count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        count_current_class = dictionary[current_class]['total_count'] + len(dictionary[current_class][j].keys())
        #current_xj_prob = count_current_class_with_value_xj / count_current_class
        current_xj_prob = np.log(count_current_class_with_value_xj)  - np.log(count_current_class)
        
        #output *= current_xj_prob
        output += current_xj_prob
        
    return output

In [ ]:
def predictSinglePoint(dictionary,x):
    classes = dictionary.keys()
    
    #best_p = -1000
    #best_class = -1
    firstRun = True
    
    for current_class in classes:
        
        if(current_class=='total_data'):
            continue
        
        p_current_class = probability(dictionary,x,current_class)
        if(firstRun or p_current_class>best_p):
            best_p = p_current_class
            best_class = current_class
        firstRun=False
            
    return best_class

In [ ]:
def predict(dictionary ,X_test):                        #result = dictionary
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    
    return y_pred
    

# Applying to iris

In [ ]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5 * second_limit
    
    for i in range(len(column)):
        if(column[i]<first_limit):
            column[i] = 0
        elif(column[i]<second_limit):
            column[i] = 1
        elif(column[i]<third_limit):
            column[i] = 2
        else:
            column[i] = 3

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
dictionary = fit(X_train,y_train)

In [ ]:
y_pred = predict(dictionary,X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))